In [1]:
from torch_geometric.datasets import QM9
from tqdm import tqdm
from simplicial_data.simplicial_data import SimplicialTransform

In [2]:
data_root = f'./datasets/QM9'
dataset = QM9(root=data_root)

In [3]:
import json
from argparse import Namespace
# Load the args from the JSON file
with open('args.json', 'r') as f:
    args = json.load(f)
args = Namespace(**args)
args.device = "cpu"

In [4]:
num_samples = args.batch_size
sample_dataset = dataset[:num_samples]
sim_args = {"dis": 0.5, "dim": 2}
transform = SimplicialTransform(dim=sim_args["dim"], dis=sim_args["dis"])
sample_dataset = [transform(sample) for sample in tqdm(sample_dataset)]

100%|██████████| 96/96 [00:00<00:00, 255.76it/s]


In [5]:
from qm9.utils import prepare_data
sample_dataset = [
    prepare_data(graph, args.target_name) for graph in tqdm(sample_dataset, desc="Preparing data")
]

Preparing data: 100%|██████████| 96/96 [00:00<00:00, 10628.02it/s]


In [6]:
def print_cell_stats(rank):
    num_cells_per_sample = [sample[f"x_{rank}"].size()[0] for sample in sample_dataset]
    max_num_cells = max(num_cells_per_sample)
    min_num_cells = min(num_cells_per_sample)
    sum_num_cells = sum(num_cells_per_sample)
    print(f"min/max/sum number of cells with rank {rank}: {min_num_cells}, {max_num_cells}, {sum_num_cells}")

print_cell_stats(0)
print_cell_stats(1)
print_cell_stats(2)

min/max/sum number of cells with rank 0: 3, 17, 950
min/max/sum number of cells with rank 1: 3, 136, 4683
min/max/sum number of cells with rank 2: 0, 0, 0


In [7]:
from torch_geometric.loader import DataLoader

# dataloaders
follow = [f"x_{i}" for i in range(args.dim + 1)] + ["x"]
sample_loader = DataLoader(
    sample_dataset,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    shuffle=True,
    follow_batch=follow,
)

In [8]:
from utils import get_model
model = get_model(args).to(args.device)

In [9]:
for batch in sample_loader:
    response = model(batch)

In [11]:
import torch
from torch_geometric.nn import global_add_pool
x = torch.rand(4,3)
b = torch.tensor([0,0,0,0])
global_add_pool(x, b)

tensor([[1.4905, 1.2863, 1.5281]])

In [13]:
response['0_0'].shape

IndexError: too many indices for tensor of dimension 1

In [ ]:
sample_dataset[2]

SimplicialComplexData(x=[3, 15], y=6.309999942779541, pos=[3, 3], z=[3], smiles='[H]O[H]', name='gdb_3', idx=[1], x_0=[3, 1], x_1=[3, 2], x_2=[0, 3], adj_0_0=[2, 6], adj_0_1=[2, 6], adj_1_1=[2, 0], adj_1_2=[2, 0], inv_0_0=[2, 6], inv_0_1=[2, 6], inv_1_1=[3, 0], inv_1_2=[3, 0])

In [ ]:
i, j = "0_1".split("_")

In [ ]:
int(j)

1